In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath("E:\Coding\my_stuff"))
from my_keys import MufexKeys # type: ignore
import numpy as np
from datetime import datetime

from strat import RSIRisingFalling
from quantfreedom.enums import CandleBodyType

from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.backtest import run_df_backtest, or_backtest

from quantfreedom.exchanges.mufex_exchange.mufex import Mufex

mufex_main = Mufex(
    api_key=MufexKeys.mainnet_neo_api_key,
    secret_key=MufexKeys.mainnet_neo_secret_key,
    use_test_net=False
)
symbol = 'BTCUSDT'

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [ ]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=3000,
)

In [ ]:
backtest_settings_tuple = BacktestSettings()

dos_tuple = DynamicOrderSettings(
    account_pct_risk_per_trade=np.array([3]),
    max_trades=np.array([4]),
    risk_reward=np.array([2, 5]),
    sl_based_on_add_pct=np.array([0.1]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5, 1.0]),
    trail_sl_when_pct=np.array([1, 2]),
)

exchange_settings_tuple = mufex_main.set_and_get_exchange_settings_tuple(
    leverage_mode=LeverageModeType.Isolated,
    position_mode=PositionModeType.HedgeMode,
    symbol=symbol,
)

static_os_tuple = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

In [ ]:
long_strat = RSIRisingFalling(
    long_short="long",
    dos_tuple=dos_tuple,
    rsi_length=np.array([14]),
    rsi_is_below=np.array([40, 60, 80]),
)
long_strat.set_entries_exits_array(
    candles=candles,
    ind_set_index=0,
)
long_strat.plot_signals(candles=candles)

In [6]:
backtest_results = run_df_backtest(
    backtest_settings_tuple=backtest_settings_tuple,
    candles=candles,
    exchange_settings_tuple=exchange_settings_tuple,
    static_os_tuple=static_os_tuple,
    strategy=long_strat,
    threads=4,
)

Error: index 12 is out of bounds for axis 0 with size 6
Error: index 6 is out of bounds for axis 0 with size 6
Error: index 18 is out of bounds for axis 0 with size 6


KeyboardInterrupt: 

In [ ]:
backtest_results.sort_values(by=["qf_score"], ascending=False).head(10)

In [ ]:
order_records_df = or_backtest(
    backtest_settings_tuple=backtest_settings_tuple,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings_tuple=exchange_settings_tuple,
    static_os_tuple=static_os_tuple,
    strategy=long_strat,
    ind_set_index=2,
    dos_index=0,
    plot_results=False,
    logger_bool=True,
)

In [ ]:
order_records_df